# Visual Assessment of Existing Maps

**Author**: Ivan Zvonkov

**Last updated**: October 30, 2023

**Description**: Generates GEE script for assessing existing cropland masks.

In [1]:
import sys
from IPython.display import IFrame

import ee
ee.Initialize()

sys.path.append("../..")

from src.compare_covermaps import TARGETS

In [2]:
country = "Senegal"

In [10]:
# Run this cell to analyze maps, computing the crop proportion for each map will take a few minutes
print("To analyze existing maps, paste the text below the dashed line into GEE\n" + "-"*100)

for i, dataset in enumerate(TARGETS.values()):
    if country not in dataset.countries:
        continue
    
    print(dataset.ee_script(country, include_prefix=(i == 0), include_export=False))
    print("\n")
    
print("""
var geom = aoi.geometry()
var geom_area = geom.area()

function printArea(image, name){
  var percentage = image.select(['crop'])
    .multiply(ee.Image.pixelArea().divide(geom_area))
    .reduceRegion({reducer: ee.Reducer.sum(), geometry: geom, scale: 100, maxPixels: 1e10});
  print("Crop % " + name + ": " + percentage.get('crop').getInfo())
}
""")

for dataset in TARGETS.values():
    if country not in dataset.countries:
        continue
    print(f"printArea({dataset.title_safe}, '{dataset.title}')")

To analyze existing maps, paste the text below the dashed line into GEE
----------------------------------------------------------------------------------------------------

var palettes = require('users/gena/packages:palettes');
var classVis = {palette: palettes.cmocean.Speed[7].slice(0,-2)}
var aoi = ee.FeatureCollection("FAO/GAUL/2015/level0")
    .filter(ee.Filter.eq('ADM0_NAME', 'Senegal'));
Map.centerObject(aoi, 7);

var copernicus = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
        .select("discrete_classification")
        .filterDate("2019-01-01", "2019-12-31").filterBounds(aoi).mosaic().clip(aoi);
copernicus = copernicus.eq(40).rename('crop')
Map.addLayer(copernicus, classVis, 'Cropland from copernicus');


var worldcover_v100 = ee.ImageCollection("ESA/WorldCover/v100").filterBounds(aoi).mosaic().clip(aoi);
worldcover_v100 = worldcover_v100.eq(40).rename('crop')
Map.addLayer(worldcover_v100, classVis, 'Cropland from worldcover-v100');


var worldcover_

Visual Assessment: https://docs.google.com/spreadsheets/d/1ZlALIwKMC3HzI8-3L4OTEYgBdNjaHKia_0oyAM6qssw/edit?usp=sharing